In [9]:
import datetime
import pandas as pd
import sqlite3
import datetime
from typing import List
from get_data import get_data_from_bs
db = sqlite3.connect('data.db')

In [22]:
from chinese_calendar import is_workday
start_time = '2020-01-02 09:30:00'
end_time = '2020-04-05 09:30:00'

In [60]:
x = pd.bdate_range(start = start_time, end = pd.to_datetime(end_time) + datetime.timedelta(days=1), freq = 'T',normalize=True).tolist()[700]

In [59]:
pd.to_datetime(end_time) + datetime.timedelta(days=1)

Timestamp('2020-04-06 09:30:00')

In [62]:
pd.bdate_range(start = start_time, end = pd.to_datetime(end_time) + datetime.timedelta(days=1), freq = '5T').tolist()[-1]

Timestamp('2020-04-06 00:00:00')

In [50]:
time(x.hour,x.minute)

datetime.time(11, 40)

In [51]:
time(12,0) >= time(x.hour,x.minute) >= time(10,30)

True

In [53]:
[x.strftime('%Y-%m-%d %H:%M:%S') for x in pd.bdate_range(start = start_time, end = end_time, freq = 'T').tolist() if is_workday(x) and (time(12,0) >= time(x.hour,x.minute) >= time(10,30) or time(15,0) >= time(x.hour,x.minute) >= time(13,00))]

['2020-01-02 10:30:00',
 '2020-01-02 10:31:00',
 '2020-01-02 10:32:00',
 '2020-01-02 10:33:00',
 '2020-01-02 10:34:00',
 '2020-01-02 10:35:00',
 '2020-01-02 10:36:00',
 '2020-01-02 10:37:00',
 '2020-01-02 10:38:00',
 '2020-01-02 10:39:00',
 '2020-01-02 10:40:00',
 '2020-01-02 10:41:00',
 '2020-01-02 10:42:00',
 '2020-01-02 10:43:00',
 '2020-01-02 10:44:00',
 '2020-01-02 10:45:00',
 '2020-01-02 10:46:00',
 '2020-01-02 10:47:00',
 '2020-01-02 10:48:00',
 '2020-01-02 10:49:00',
 '2020-01-02 10:50:00',
 '2020-01-02 10:51:00',
 '2020-01-02 10:52:00',
 '2020-01-02 10:53:00',
 '2020-01-02 10:54:00',
 '2020-01-02 10:55:00',
 '2020-01-02 10:56:00',
 '2020-01-02 10:57:00',
 '2020-01-02 10:58:00',
 '2020-01-02 10:59:00',
 '2020-01-02 11:00:00',
 '2020-01-02 11:01:00',
 '2020-01-02 11:02:00',
 '2020-01-02 11:03:00',
 '2020-01-02 11:04:00',
 '2020-01-02 11:05:00',
 '2020-01-02 11:06:00',
 '2020-01-02 11:07:00',
 '2020-01-02 11:08:00',
 '2020-01-02 11:09:00',
 '2020-01-02 11:10:00',
 '2020-01-02 11:

In [10]:
stock_code = "sz.000001"
start_date = "2020-01-01"
end_date = "2020-12-31"
freq = '5'
adjustflag = '1'
result = get_data_from_bs(stock_code, start_date = start_date, end_date = end_date, frequency = freq, adjustflag= adjustflag)

login success!
login respond error_code:0
login respond  error_msg:success
query_history_k_data_plus respond error_code:0
query_history_k_data_plus respond  error_msg:success
Get data successfully!
logout success!


In [7]:
class DataAgent:
    def __init__(self, symbols: List):
        # data dict
        self.kline_history_backtest = {}  # the keys is datetime and every datetime is a dict which keys are symbols and the value are also dicts containing data
        self.latest_data = {}
        self.symbols = symbols
        self.datetimelist = []
        self.datetimeiter = iter([])

    def get_all_data(self, use_frequency: str, start_time: str, end_time: str):

        # get trading datetimelist:could be change according to other time interval data
        if use_frequency == "1":
            self.datetimelist = [datetime.datetime.strptime(str(x), '%Y-%m-%d %H:%M:%S') for x in
                                 pd.date_range(start = start_time, end = end_time, freq = 'T').tolist()]
        elif use_frequency == "5":
            self.datetimelist = [datetime.datetime.strptime(str(x), '%Y-%m-%d %H:%M:%S') for x in
                                 pd.date_range(start = start_time, end = end_time, freq = '5T').tolist()]
        elif use_frequency == "60":
            self.datetimelist = [datetime.datetime.strptime(str(x), '%Y-%m-%d %H:%M:%S') for x in
                                 pd.date_range(start = start_time, end = end_time, freq = '60T').tolist()]
        '''
        get the history kline data of symbol from start_time to end_time with use_frequency :now only could choose from "1m" or "5m"

        :params symbol: code
        :params use_frequency: choose from '1m' or '5m'
        :params start_time : starting time to backtesting eg: '2018-01-14 08:00:00'
        :params end_time : ending time to backtesting eg: '2018-01-14 08:00:00'
        '''

        cursor = db.cursor()

        # fetch the data in symbols one by one
        for symbol in self.symbols:
            sql = f"SELECT * FROM '{use_frequency}' where code = '{symbol}' and time >= '{start_time}' and time <= '{end_time}'"
            cursor.execute(sql)
            data_set = cursor.fetchall()

            if len(data_set) == 0:
                print("No data!")
                raise Exception
            
            # ['index','date','time','code','open','high','low','close','volume','amount','adjustflag'])
          

            # similar to the live data but filter something not used every date, keys of self.kline_history_backtest is datetime
            for i in range(len(data_set)):
                if data_set[i][2] not in self.kline_history_backtest.keys():  # time
                    self.kline_history_backtest[data_set[i][2]] = {}

                self.kline_history_backtest[data_set[i][2]][symbol] = []
                self.kline_history_backtest[data_set[i][2]][symbol] = {"e": "kline", "E": data_set[i][2], "s": symbol,
                                                                       "k": {"o": data_set[i][4], "h": data_set[i][5],
                                                                             "l": data_set[i][6], "c": data_set[i][7],
                                                                             "v": data_set[i][8]}}
        cursor.close()
        db.close()

    def get_market_price_trade(self, time: datetime.date, symbol, delay_min: int = 1):
        '''

        if at time T generate the buy or sell signal, then buy or sell at T+n at its close price
        return the trading price and trading time

        :params delay_min: consider the delay, trading at T+n minute

        return the trade price and the trade time
        '''

        try:
            price = self.kline_history_backtest[time + datetime.timedelta(minutes = delay_min)][symbol]["k"]["c"]
            return price, time + datetime.timedelta(minutes = delay_min)
        except:
            print("No price data.")
            raise ValueError

    def get_market_price_now(self, time: datetime.datetime, symbol):
        '''
        get the latest close price

        '''
        price = self.kline_history_backtest[time][symbol]["k"]["c"]
        return price

    def generate_backtest_datetime_iter(self, start_time: datetime.datetime, end_time: datetime.datetime):
        try:
            self.datetimeiter = iter(
                self.datetimelist[self.datetimelist.index(start_time):self.datetimelist.index(end_time) + 1])
        except ValueError:
            print("Historical data couldn't cover this time span!")

    def _get_new_datetime(self):
        return next(self.datetimeiter)

    def get_latest_use_data(self, use_symbol_list, n = 1):
        use_symbol_data = {}
        if use_symbol_list is not None:
            for symbol in use_symbol_list:
                try:  # in case for symbol is not valid
                    use_symbol_data[symbol] = []
                    try:  # in case couldn't get n historical data
                        use_symbol_data[symbol] = self.latest_data[symbol][-n:]
                    except:
                        continue
                except KeyError:
                    print("{symbol} is not a valid symbol.").format(symbol = symbol)

            return use_symbol_data

    def update_data(self):
        '''
        return the update symbol list and the time
        '''
        try:
            datatime = self._get_new_datetime()
            print(datatime)
            if datatime in self.kline_history_backtest.keys():
                update_symbol_list = list(self.kline_history_backtest[datatime].keys())
                for symbol in update_symbol_list:
                    if symbol not in self.latest_data.keys():
                        self.latest_data[symbol] = []
                        self.latest_data[symbol].append(self.kline_history_backtest[datatime][symbol])
                    else:
                        self.latest_data[symbol].append(self.kline_history_backtest[datatime][symbol])
                return update_symbol_list, datatime  # so the strategy will generate action of this symbols
            else:  # if at this time no  data to update due to some errors
                return None, datatime  # that means no update this time

        except StopIteration:  # no data
            print("Backtesting done!")
            return None, None  # so if date_time is None ,means backtest done

In [3]:
dh = DataAgent(['sz.000001'])

In [8]:
dh.get_all_data('60', '2020-01-02', '2020-01-05')